# Oanda Demo Trading Notebook

## Packages & Tools

In [1]:
from utils.packages import *
from utils.tools import *

## Tools

In [2]:
def make_order(accountID, stop_price, instrument, units, order_type):
    stopLossOnFill = StopLossDetails(price=stop_price)
    
    if order_type == 'long':
        units = units * 1
    elif order_type == 'short':
        units = units * (-1)

    ordr = MarketOrderRequest(
        instrument = instrument,
        units=units,
        stopLossOnFill=stopLossOnFill.data)

    r = orders.OrderCreate(accountID, data=ordr.data)
    rv = api.request(r)
    return(rv)

In [3]:
def close_order(accountID, order_type, instrument):
    data_long = {"longUnits": "ALL"}
    data_short = {"shortUnits": "ALL"}
    
    if order_type == 'long':
        data = data_long
    elif order_type == 'short':
        data = data_short
        
    r = positions.PositionClose(accountID=accountID,
                                instrument=instrument,
                                data=data)
    rv = api.request(r)
    return(rv)

## API Setup

Read from config file

In [4]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currencies = yaml.load(temp_file)

temp_file = 'config/stepped_loss.yaml'
with open(temp_file) as temp_file:
    loss_limits = yaml.load(temp_file)
    
temp_file = 'config/wma.yaml'
with open(temp_file) as temp_file:
    wma = yaml.load(temp_file)    

In [5]:
instrument = wma['parameter']['instrument']
short_moving_window = wma['parameter']['short_moving_window']
long_moving_window = wma['parameter']['long_moving_window']
lists_size = wma['parameter']['lists_size']

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']
api = API(access_token = access_token)

## Code Engine

In [6]:
'''instrument="EUR_USD"
short_moving_window = 2
long_moving_window = 4
lists_size = 10'''

'instrument="EUR_USD"\nshort_moving_window = 2\nlong_moving_window = 4\nlists_size = 10'

In [7]:
loss_limit_num = 25

pip = 0.0001
loss_limit = loss_limit_num * pip

In [8]:
params = {'instruments': instrument}
order_flag = 'not_ordered'
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

short_wma = 0
short_flag = True
short_list = collections.deque([])
short_wma_list = collections.deque([])
short_weights = get_weights(short_moving_window)

long_wma = 0
long_flag = True
long_list = collections.deque([])
long_wma_list = collections.deque([])
long_weights = get_weights(long_moving_window)

tick_list = collections.deque([])

long_list_counter = 0
list_ready = False
old_direction =  'none'

order_flag = False

profit_target = 0.25 # pip target of actual price
price_allowed_buffer = 0.15 # percentage of SWMA decrease

profit_target = profit_target * pip 

pl_list = collections.deque([])
consecutive_losses = 2
negative_run =  False
pl_pip = 0

max_gap_wma = 0
max_gap_wma_threshold = 1
max_gap_wma_threshold = max_gap_wma_threshold * pip

valid_iteration = False

units = 1

first_position_changed = False

initial_position_captured = False

for i, resp in tqdm(enumerate(rv)):        
    resp_type = resp['type']       

    if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        pass

    #-----------------------------------------------------------------------------------------------------
    elif resp_type == 'PRICE': # Check if we are yet to make the order               
        date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
        sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response  


    #-----------------------------------------------------------------------------------------------------
        #Short list prep
        #-----------------------------------------------------------------------------------------------------
        if len(short_list) < short_moving_window:
            short_list.append(tick_price)

        if short_flag and len(short_list) == short_moving_window:
            short_wma = sum(np.array(short_list)*short_weights)
            short_flag =False

        if short_flag == False and len(short_list) == short_moving_window:
            short_list.popleft()   
            short_list.append(tick_price) 
            short_wma = sum(np.array(short_list)*short_weights)



        #Long list prep   
        #-----------------------------------------------------------------------------------------------------                
        if len(long_list) < long_moving_window:
            long_list.append(tick_price)

        if long_flag and len(long_list) == long_moving_window:
            long_wma = sum(np.array(long_list)*long_weights)
            long_flag = False

        if long_flag == False and len(long_list) == long_moving_window:
            long_list.popleft()   
            long_list.append(tick_price) 
            long_wma = sum(np.array(long_list)*long_weights)   
            long_list_counter += 1
            if long_list_counter >= lists_size:
                list_ready = True
    #-----------------------------------------------------------------------------------------------------
        #Wrong strategy this skips iterations while holding a position
        gap_wma = abs(long_wma - short_wma)
        max_gap_wma = max(max_gap_wma, gap_wma)
        
        print(f'long_wma:{long_wma}, short_wma:{short_wma}, gap_wma:{gap_wma}, max_gap_wma:{max_gap_wma}, max_gap_wma_threshold:{max_gap_wma_threshold}')
        if max_gap_wma > max_gap_wma_threshold:
            valid_iteration = True
        
        #----------------------------------------------------------------------------------------------------- 
        if valid_iteration == False and list_ready:
            #print(f'Gap: {max_gap_wma}, Threshold: {max_gap_wma_threshold}')
            pass
            
        elif valid_iteration and list_ready:
            if list_ready:
                if short_wma > long_wma:
                    current_position = 'positive'
                elif short_wma < long_wma:
                    current_position = 'negative'
                elif short_wma == long_wma:
                    current_position = 'same level'

            if list_ready and initial_position_captured == False:
                initial_position = current_position
                print(f'initial_position : {initial_position}')
                initial_position_captured =  True
                winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC) 

            if list_ready and initial_position_captured:
                if initial_position == current_position:
                    pass

                elif current_position == 'same level':
                    print('Same level, wait for next iteration')

                elif initial_position != current_position and current_position != 'same level':
                    print(f'initial_position:{initial_position}')
                    print(f'current_position:{current_position}')

                    #Select order type -------------------------------------------------------------------------
                    if current_position == 'positive':
                        old_order_type = 'short'
                        order_type = 'long'
                        stop_price = buy_price - loss_limit                

                    elif current_position == 'negative':
                        old_order_type = 'long'
                        order_type = 'short'
                        stop_price = sell_price + loss_limit                

                    # Ordering -------------------------------------------------------------------------    

                    if order_flag:
                        print(f'Closing {old_order_type} positions if any')
                        close_order_log = close_order(accountID, old_order_type ,instrument)
                        order_flag = False
                        max_gap_wma = 0

                    print(f'Going {order_type}')
                    make_order_log = make_order(accountID, stop_price, instrument, units, order_type)
                    print('ordered')
                    first_long =  True
                    order_flag = True

                    initial_position = current_position


                if initial_position == current_position and current_position != 'same level' and order_flag:
                    if order_type == 'long':     
                        ordered_buy_price = float(make_order_log['orderFillTransaction']['fullPrice']['asks'][0]['price'])
                        profit = sell_price - ordered_buy_price 
                        
                        if first_long:
                            max_price = short_wma
                            first_long = False

                        max_price = max(short_wma, max_price)
                        buffered_max_price = max_price - (max_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma <= buffered_max_price:    
                            close_order_log = close_order(accountID, order_type ,instrument)                        
                            order_flag = False
                            max_gap_wma = 0
                            close_reason = "Take_profit"
                            print(close_reason)
                            
                            
                    if order_type == 'short':     
                        ordered_sell_price = float(make_order_log['orderFillTransaction']['fullPrice']['bids'][0]['price'])
                        profit = ordered_sell_price - buy_price                
                        
                        if first_long:
                            min_price = short_wma
                            first_long = False

                        min_price = min(short_wma, min_price)
                        buffered_min_price = min_price + (min_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma <= buffered_min_price:    
                            close_order_log = close_order(accountID, order_type ,instrument)                        
                            order_flag = False
                            max_gap_wma = 0
                            close_reason = "Take_profit"
                            print(close_reason)                    

255it [04:46,  3.01it/s]

initial_position : negative


343it [05:32,  1.52it/s]

initial_position:negative
current_position:positive
Going long


344it [05:32,  1.46it/s]

...

initial_position:positive
current_position:negative
Closing long positions if any
Going short


508it [07:23,  1.14s/it]

ordered


524it [07:37,  1.15it/s]


KeyboardInterrupt: 